# Building a Spam Filter with Naive Bayes

## Introduction
In this project we will build a spam filter using a multinomial Naive Bayes algorithm. This means that we will we will use data with human classifications to train the algorithm to detect spam.

The data set comprises 5574 instances of spam and ham (non-spam) messages collated from multiple resources by Tiago A. Almeida and José María Gómez Hidalg and can be downloaded from [this link](https://dq-content.s3.amazonaws.com/433/SMSSpamCollection). There is more information about how the data was sourced [here](https://archive.ics.uci.edu/dataset/228/sms+spam+collection).

##  Exploring the Dataset

In [1]:
import pandas as pd

file = 'SMSSpamCollection'

data = pd.read_csv(file, sep='\t', header=None, names=['Label', 'SMS'])

data.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [2]:
data.tail()

,Label,SMS
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...
5571,ham,Rofl. Its true to its name


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Label   5572 non-null   object
 1   SMS     5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [4]:
# Proportions of data set as percentages
data['Label'].value_counts(normalize=True)*100

ham     86.593683
spam    13.406317
Name: Label, dtype: float64

## Training and Test Set

It's important when building a tool like this that we retain a portion of the data to test the tool. This data should be separate from the data we train the algorithm.  So we need to split our data into:
- a training set
- a test set

A typical ratio for this would by 80/20, since we want to train the algorithm on as much data as possible. We need the other 20% to test becuase this data has already been classified as spam or ham, meaning that we can compare the ouput from our tool against the classifications and actually measure how well the tool is working.

We split the data set like this:
- The training set will have 4,458 messages (about 80% of the dataset).
- The test set will have 1,114 messages (about 20% of the dataset).

With the tool we are aiming for an accuracy greater than 80%.

In [5]:
# Randomize the dataset
data_random = data.sample(frac=1, random_state=1)

# Calculate index for split
training_test_index = round(len(data_random) * 0.8)

# Training/Test split
training = data_random[:training_test_index].reset_index(drop=True)
test = data_random[training_test_index:].reset_index(drop=True)

print(training.shape)
print(test.shape)

(4458, 2)
(1114, 2)


In [6]:
# Proportions of training set as percentages
training['Label'].value_counts(normalize=True)*100

ham     86.54105
spam    13.45895
Name: Label, dtype: float64

In [7]:
# Proportions of test set as percentages
test['Label'].value_counts(normalize=True)*100

ham     86.804309
spam    13.195691
Name: Label, dtype: float64

We first randomized the whole data set and then split the set into to parts rows 0 to 4457 formed a new training data set and rows 4458 to 5573 became the test set. Looking at the proportions of ham and spam, we can see that they are close to that of the complete data set <0.5% difference.

## Data Cleaning

The classification that we want to make is based on the proportionality of these two equations from naives Bayes theory:

$$
  P(Spam|w_{1}, w_{2},...w_{n})\propto P(Spam)\cdot\prod_{i=1}^n P(w_i|Spam)
$$
$$
  P(Ham|w_{1}, w_{2},...w_{n})\propto P(Ham)\cdot\prod_{i=1}^n P(w_i|Ham)
$$

In order to calculate P(w<sub>i</sub>|Spam) and P(w<sub>i</sub>|Ham), we need to use the following:
$$
  P(w_i|Spam) = \frac{N_{w_{i}|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}
$$
$$
  P(w_i|Ham) = \frac{N_{w_{i}|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
$$
Where:
$$
N_{w_{i}|Spam} = \text{the number of times the word }w_{i}\text{ occurs in spam messages}
$$
$$
N_{w_{i}|Ham} = \text{the number of times the word }w_{i}\text{ occurs in non-spam messages}
$$
$$
N_{Spam} = \text{total number of words in spam messages}
$$
$$
N_{Ham} = \text{total number of words in non-spam messages}
$$
$$
N_{Vocabulary} = \text{total number of words in vocabulary}
$$
$$
\alpha = 1 \quad\text{(}\alpha\text{ is a smoothing parameter)}
$$

### Letter Case and Punctuation

In order to reach this point we need to format our data differently. We can start by removing punctuation and converting everything to lowercase.

In [8]:
training.head()

,Label,SMS
0,ham,"Yep, by the pretty sculpture"
1,ham,"Yes, princess. Are you going to make me moan?"
2,ham,Welp apparently he retired
3,ham,Havent.
4,ham,I forgot 2 ask ü all smth.. There's a card on ...


In [9]:
training['SMS'] = training['SMS'].str.replace('\W', ' ')
training['SMS'] = training['SMS'].str.lower()

training.head()

/var/folders/w6/bggh9nlx1wx9th6t6ctvwrd00000gn/T/ipykernel_31990/1524640398.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  training['SMS'] = training['SMS'].str.replace('\W', ' ')


,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...


### Creating the Vocabulary

We now need to create our vocabulary set so that we end up with something that looks like the image below:

![Building a vocabulary](cpgp_dataset_3.png)

We need to create a list of unique words from the SMS column.

In [10]:
training['SMS'] = training['SMS'].str.split()

vocabulary = []
for sms in training['SMS']:
    for word in sms:
        vocabulary.append(word)

print("All words: "+str(len(vocabulary)))        
vocabulary = list(set(vocabulary))
print("Unique words: "+str(len(vocabulary)))


All words: 72427
Unique words: 7783


In [11]:
training.head()

,Label,SMS
0,ham,"[yep, by, the, pretty, sculpture]"
1,ham,"[yes, princess, are, you, going, to, make, me,..."
2,ham,"[welp, apparently, he, retired]"
3,ham,[havent]
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,..."


### Final Training Set

We can now construct a dictionary using our vocabulary, to create a data frame that has a count of each word from the vocabulary for each sms.

In [12]:
word_counts_per_sms = {unique_word: [0] * len(training['SMS']) for unique_word in vocabulary}

for index, sms in enumerate(training['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1

In [13]:
word_counts_per_sms = pd.DataFrame(word_counts_per_sms)

In [14]:
word_counts_per_sms.head()

,hearted,2day,intention,upping,pls,veggie,flood,velly,album,glasgow,...,hellogorgeous,fizz,wrld,exciting,blonde,slowly,semiobscure,tells,tones2you,vegetables
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
word_counts_per_sms.shape

(4458, 7783)

In [16]:
training_set = pd.concat([training, word_counts_per_sms], axis=1, join='inner')

In [17]:
training_set.head()

,Label,SMS,hearted,2day,intention,upping,pls,veggie,flood,velly,...,hellogorgeous,fizz,wrld,exciting,blonde,slowly,semiobscure,tells,tones2you,vegetables
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
training_set.shape

(4458, 7785)

## Building the Algorithm

### Calculating Constants First

The constants in our equations are the following:
- $P(Ham)$ is the probability of picking ham from the classified data set: $\frac{Number Ham Messages}{Total Messages}$
- $P(Spam)$ is the probability of picking spam from the classified data set: $\frac{Number Spam Messages}{Total Messages}$
- $N_{Ham}$ is the number of words across all ham messages
- $N_{Spam}$ is the number of words across all spam messages
- $N_{Vocabulary}$ is the number of unique words

In [21]:
ham_messages = training_set[training_set['Label'] == 'ham']
spam_messages = training_set[training_set['Label'] == 'spam']

# Calculate the probabilities using the above formulas
p_ham = len(ham_messages) / len(training_set)
p_spam = len(spam_messages) / len(training_set)


In [22]:
# n_ham
n_words_per_ham_message = ham_messages['SMS'].apply(len)
n_ham = n_words_per_ham_message.sum()

# n_spam
n_words_per_spam_message = spam_messages['SMS'].apply(len)
n_spam = n_words_per_spam_message.sum()

# n_vocab
n_vocab = len(vocabulary)

# alpha
alpha = 1

print('P(Spam): ', p_spam)
print('P(Ham): ', p_ham)
print('n_spam: ', n_spam)
print('n_ham: ', n_ham)
print('n_vocab: ', n_vocab)

P(Spam):  0.13458950201884254
P(Ham):  0.8654104979811574
n_spam:  15190
n_ham:  57237
n_vocab:  7783


### Calculating Parameters

In [27]:
p_word_given_ham = {unique_word: 0 for unique_word in vocabulary}
p_word_given_spam = {unique_word: 0 for unique_word in vocabulary}

for word in vocabulary:
    # calculate for ham
    n_word_given_ham = ham_messages[word].sum()
    p_word_given_ham[word] = (n_word_given_ham + alpha) / (n_ham + alpha*n_vocab)  

    # calculate for spam
    n_word_given_spam = spam_messages[word].sum()
    p_word_given_spam[word] = (n_word_given_spam + alpha) / (n_spam + alpha*n_vocab)

In [37]:
# test for the word 'himself'
print('P(himself|spam): ', p_word_given_spam['himself'])
print('P(himself|ham): ', p_word_given_ham['himself'],'\n')


# test for the word 'secret'
print('P(secret|spam): ', p_word_given_spam['secret'])
print('P(secret|ham): ', p_word_given_ham['secret'], '\n')

# test for the word 'hi'
print('P(hey|spam): ', p_word_given_spam['hi'])
print('P(hey|ham): ', p_word_given_ham['hi'])

P(himself|spam):  4.3529360553693465e-05
P(himself|ham):  3.075976622577668e-05 

P(secret|spam):  0.0003482348844295477
P(secret|ham):  6.151953245155337e-05 

P(hey|spam):  0.0006964697688590954
P(hey|ham):  0.0016456474930790525


##  Classifying A New Message

In [29]:
import re

def classify(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

     # initialise variables for our classifying probabilities
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    for word in message:
        if word in p_word_given_spam:
            p_spam_given_message *= p_word_given_spam[word]
            
        if word in p_word_given_ham:
            p_ham_given_message *= p_word_given_ham[word]


    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

In [30]:
# Test the algorithm
classify('WINNER!! This is the secret code to unlock the money: C3421.')

P(Spam|message): 1.3481290211300841e-25
P(Ham|message): 1.9368049028589875e-27
Label: Spam


In [31]:
classify("Sounds good, Tom, then see u there")

P(Spam|message): 2.4372375665888117e-25
P(Ham|message): 3.687530435009238e-21
Label: Ham


## Measuring the Spam Filter's Accuracy

In [40]:
def classify_test_set(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in p_word_given_spam:
            p_spam_given_message *= p_word_given_spam[word]

        if word in p_word_given_ham:
            p_ham_given_message *= p_word_given_ham[word]

    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_spam_given_message > p_ham_given_message:
        return 'spam'
    else:
        return 'needs human classification'

In [41]:
test['predicted'] = test['SMS'].apply(classify_test_set)
test.head()

,Label,SMS,predicted
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


In [50]:
correct = 0
total = len(test)

for index,row in test.iterrows():
    if row['Label'] == row['predicted']:
        correct+=1

    
accuracy = correct / total
print('accuracy: ', accuracy*100)

accuracy:  98.74326750448833


We can look at some other measure if we re config this as spam an not spam - spam as a positive identification 
- TP = correctly labelled spam
- FP = incorrectly labelled spam
- TN = correctly labelled non-spam
- FN = incorrectly labelled non-spam

In [68]:
tp=0
fp=0
tn=0
fn=0

for index, row in test.iterrows():
    if (row['Label'] == 'spam') & (row['predicted'] == 'spam'):
        tp+=1
    if (row['Label'] == 'ham') & (row['predicted'] == 'spam'):
        fp+=1
    if (row['Label'] == 'ham') & (row['predicted'] == 'ham'):
        tn+=1
    if (row['Label'] == 'spam') & (row['predicted'] == 'ham'):
        fn+=1
        
accuracy = (tp+tn) / total
precision = tp / (tp+fp)
recall = tp / (tp+fn)
f1 = 2*(precision*recall) / (precision+recall)

print('accuracy: {:.2%}\n'.format(accuracy))
print('“Out of all the positive predictions we made, how many were true?”')
print('precision: {:.2%} \n'.format(precision))
print('“Out of all the data points that should be predicted as true, how many did we correctly predict as true?”')
print('recall: {:.2%}\n'.format(recall))
print('We combine recall and precision in an f1 sco')
print('f1: {:.2%}\n'.format(f1))

accuracy: 98.74%

“Out of all the positive predictions we made, how many were true?”
precision: 96.53% 

“Out of all the data points that should be predicted as true, how many did we correctly predict as true?”
recall: 94.56%

f1: 95.53%

